# Laboratorio Completo en SQLite y Flask
Este notebook interactivo organiza paso a paso el laboratorio práctico con explicaciones y bloques de código que puedes copiar y ejecutar.

## 1. Inicialización y Carga de Datos
En este apartado creamos la base de datos SQLite, definimos la tabla de usuarios, la tabla de auditoría y los triggers para registrar cada operación.

In [ ]:
# db_init.py
import sqlite3

DB_PATH = 'usuarios.db'

def init_db():
    # Conexión y cursor
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    # Creación de tabla usuarios
    c.execute('''
        CREATE TABLE IF NOT EXISTS usuarios (
            id       INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre   TEXT NOT NULL,
            correo   TEXT NOT NULL,
            rol      TEXT NOT NULL
        );
    ''')
    # Creación de tabla de auditoría
    c.execute('''
        CREATE TABLE IF NOT EXISTS audit_log (
            id          INTEGER PRIMARY KEY AUTOINCREMENT,
            tabla       TEXT,
            operacion   TEXT,
            registro_id INTEGER,
            timestamp   DATETIME DEFAULT CURRENT_TIMESTAMP
        );
    ''')
    # Triggers para auditoría de INSERT, UPDATE y DELETE
    c.execute('''
        CREATE TRIGGER IF NOT EXISTS trg_usuarios_insert
        AFTER INSERT ON usuarios
        BEGIN
            INSERT INTO audit_log(tabla, operacion, registro_id)
            VALUES('usuarios','INSERT',NEW.id);
        END;
    ''')
    c.execute('''
        CREATE TRIGGER IF NOT EXISTS trg_usuarios_update
        AFTER UPDATE ON usuarios
        BEGIN
            INSERT INTO audit_log(tabla, operacion, registro_id)
            VALUES('usuarios','UPDATE',NEW.id);
        END;
    ''')
    c.execute('''
        CREATE TRIGGER IF NOT EXISTS trg_usuarios_delete
        AFTER DELETE ON usuarios
        BEGIN
            INSERT INTO audit_log(tabla, operacion, registro_id)
            VALUES('usuarios','DELETE',OLD.id);
        END;
    ''')
    # Inserción de datos de ejemplo
    usuarios = [
        ("María Pérez", "maria.perez@example.com", "Analista"),
        ("Juan González", "juan.gonzalez@example.com", "Pentester"),
        ("Ana Torres",   "ana.torres@example.com",   "Auditor"),
        ("Carlos Soto",  "carlos.soto@example.com",  "Administrador")
    ]
    c.executemany(
        'INSERT OR IGNORE INTO usuarios (nombre, correo, rol) VALUES (?,?,?)',
        usuarios
    )
    # Guardar cambios y cerrar
    conn.commit()
    conn.close()
    print("Base de datos inicializada y datos de ejemplo insertados.")

# Ejecutar inicialización
init_db()

## 2. Funciones CRUD con Filtrado, Orden y Paginación
Estas funciones permiten listar usuarios aplicando filtros por rol, ordenar resultados y paginar la salida.

In [ ]:
# crud.py
import sqlite3
from typing import List, Tuple, Optional

DB_PATH = 'usuarios.db'

def get_usuarios(
    rol_filter: Optional[str] = None,
    order_by: str = 'id',
    asc: bool = True,
    page: int = 1,
    per_page: int = 10
) -> List[Tuple]:
    offset = (page - 1) * per_page
    base_sql = "SELECT id, nombre, correo, rol FROM usuarios"
    params = []
    if rol_filter:
        base_sql += " WHERE rol = ?"
        params.append(rol_filter)
    base_sql += f" ORDER BY {order_by} {'ASC' if asc else 'DESC'}"
    base_sql += " LIMIT ? OFFSET ?"
    params += [per_page, offset]

    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    c.execute(base_sql, params)
    rows = c.fetchall()
    conn.close()
    return rows

# Ejemplos de uso
print("Usuarios página 1 (2 por página):", get_usuarios(page=1, per_page=2))
print("Usuarios con rol 'Pentester':", get_usuarios(rol_filter="Pentester"))

## 3. Exportación a CSV y JSON
Exportamos los registros a formatos CSV y JSON para análisis externo.

In [ ]:
# export.py
import sqlite3, csv, json

DB_PATH = 'usuarios.db'

def export_csv(path='usuarios.csv'):
    conn = sqlite3.connect(DB_PATH)
    rows = conn.execute("SELECT id, nombre, correo, rol FROM usuarios").fetchall()
    conn.close()
    with open(path, 'w', newline='', encoding='utf-8') as f:
        w = csv.writer(f)
        w.writerow(['id','nombre','correo','rol'])
        w.writerows(rows)
    print(f"Exportado a CSV: {path}")

def export_json(path='usuarios.json'):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.execute("SELECT id, nombre, correo, rol FROM usuarios")
    cols = [d[0] for d in cursor.description]
    rows = cursor.fetchall()
    conn.close()
    data = [dict(zip(cols, row)) for row in rows]
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"Exportado a JSON: {path}")

# Ejecutar exportaciones
export_csv()
export_json()

## 4. Demostración de SQL Injection Vulnerable
Se muestra cómo la concatenación insegura permite burlar consultas.

In [ ]:
# vulnerable_query.py
import sqlite3

DB_PATH = 'usuarios.db'

def vulnerable_lookup(rol: str):
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    sql = f"SELECT id, nombre FROM usuarios WHERE rol = '{rol}';"
    for row in c.execute(sql):
        print(row)
    conn.close()

print("Lookup legítimo:")
vulnerable_lookup("Pentester")
print("\nLookup con inyección:")
vulnerable_lookup("Pentester' OR '1'='1")

## 5. Consulta Segura con Prepared Statements
La versión segura usa parámetros para evitar inyección.

In [ ]:
# secure_query.py
import sqlite3

DB_PATH = 'usuarios.db'

def secure_lookup(rol: str):
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    c.execute("SELECT id, nombre FROM usuarios WHERE rol = ?", (rol,))
    for row in c.fetchall():
        print(row)
    conn.close()

print("Lookup seguro:")
secure_lookup("Pentester")
print("\nIntento de inyección seguro:")
secure_lookup("Pentester' OR '1'='1")

## 6. Aplicación Flask con Control de Roles
Levantamos un servidor Flask que gestiona login, dashboard y un panel administrativo accesible solo a administradores.

In [ ]:
# app_flask.py
from flask import Flask, request, session, redirect, render_template_string
import sqlite3

app = Flask(__name__)
app.secret_key = 'supersecretkey'
DB_PATH = 'usuarios.db'

def query_user(username, password):
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    c.execute("SELECT id, nombre, rol FROM usuarios WHERE nombre = ? AND correo = ?", (username, password))
    user = c.fetchone()
    conn.close()
    return user

def get_usuarios():
    conn = sqlite3.connect(DB_PATH)
    rows = conn.execute("SELECT id, nombre, correo, rol FROM usuarios").fetchall()
    conn.close()
    return rows

@app.route('/login', methods=['GET','POST'])
def login():
    if request.method == 'POST':
        uname = request.form['username']
        pwd   = request.form['password']
        user = query_user(uname, pwd)
        if user:
            session['user_id'], session['user_name'], session['user_role'] = user
            return redirect('/dashboard')
        return 'Credenciales inválidas', 401
    return render_template_string('''
        <form method="post">
            Usuario: <input name="username"><br>
            Correo: <input name="password"><br>
            <button>Ingresar</button>
        </form>
    ''')

@app.route('/dashboard')
def dashboard():
    if 'user_id' not in session:
        return redirect('/login')
    return render_template_string('''
        <h1>Bienvenido, {{name}} ({{role}})</h1>
        <a href="/admin">Panel Admin</a> | <a href="/logout">Cerrar sesión</a>
    ''', name=session['user_name'], role=session['user_role'])

@app.route('/admin')
def admin_panel():
    if session.get('user_role') != 'Administrador':
        return 'Acceso denegado', 403
    table = ''.join(f"<tr><td>{u[0]}</td><td>{u[1]}</td><td>{u[2]}</td><td>{u[3]}</td></tr>" for u in get_usuarios())
    return render_template_string('''
        <h1>Panel Admin</h1>
        <table border=1><tr><th>ID</th><th>Nombre</th><th>Correo</th><th>Rol</th></tr>{{rows}}</table>
    ''', rows=table)

@app.route('/logout')
def logout():
    session.clear()
    return redirect('/login')

if __name__ == '__main__':
    app.run(debug=True)